In [18]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [19]:
# Create an engine to a database file called `hawaii.sqlite`
engine = create_engine("sqlite:///hawaii.sqlite")

In [20]:
# Create a connection to the engine called `conn`
conn = engine.connect()

In [21]:
# Use `declarative_base` from SQLAlchemy to model the measurements table as an ORM class

Base = declarative_base()

class Measurements(Base):
    __tablename__ = 'measurements'

    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Integer)
    
class Stations(Base):
    __tablename__ = 'stations'
    
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)

In [22]:
# Use `create_all` to create the demographics table in the database
Base.metadata.create_all(engine)

In [23]:
import pandas as pd

In [24]:
# Load the cleaned csv file into a pandas dataframe
clean_measurements_df = pd.read_csv("clean_hawaii_measurements.csv")
clean_stations_df = pd.read_csv("clean_hawaii_stations.csv")

In [25]:
# Use Orient='records' to create a list of data to write
measurement_data = clean_measurements_df.to_dict(orient='records')
stations_data = clean_stations_df.to_dict(orient='records')
stations_data[0]

{'Unnamed: 0': 0,
 'elevation': 3.0,
 'latitude': 21.2716,
 'longitude': -157.8168,
 'name': 'WAIKIKI 717.2, HI US',
 'station': 'USC00519397'}

In [26]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [27]:
# Save the reference to tables as variables
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [28]:
# Use `table.delete()` to remove any existing data.
conn.execute(measurements_table.delete())
conn.execute(stations_table.delete())

In [29]:
# Use `table.insert()` to insert the data into the table
conn.execute(measurements_table.insert(), measurement_data)
conn.execute(stations_table.insert(), stations_data)

In [30]:
# Test that the insert works by fetching the first 5 rows. 
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [31]:
from sqlalchemy.ext.automap import automap_base

In [32]:
# Reflect Database into ORM class
Base = automap_base()
Base.prepare(engine, reflect=True)

In [33]:
Base.classes.measurements
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]